In [5]:
!pip install lxml

     |████████████████████████████████| 4.5 MB 44 kB/s eta 0:00:016
You should consider upgrading via the '/opt/miniconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
from datetime import date,timedelta
import requests
import json
from zipfile import ZipFile
import os
import sys
import pandas as pd
import glob
import numpy as np
# XBRLをpython形式に変換するライブラリのフォルダパス
sys.path.append(r'./data/input/xbrl_reader_for_edinet')
from xbrl_proc import read_xbrl_from_zip
import urllib3
from urllib3.exceptions import InsecureRequestWarning
urllib3.disable_warnings(InsecureRequestWarning)

In [38]:
def get_list(start,end,company_df):
    '''指定した期間、報告書種類、会社で報告書を取得し、取得したファイルのパスの辞書を返す'''
    #取得期間の日付リストを作成
    day_term = [start + timedelta(days=i) for i in range((end - start).days)]
    
    # 会社コードリストを取得
    company_list =list(company_df['ＥＤＩＮＥＴコード'])
    
    #データ抜出時に使用する、有価証券報告書および四半期報告書のコードの設定
    ordinance_code = "010"
    form_code_quart ="043000" # 四半期報告書
    form_code_securities ="030000" #有価証券報告書
    
    # EDINETのAPIで、書類一覧を取得し、各日ごとに必要な書類の項目を抜き出し
    quart_list =[] # 四半期報告書のリスト
    securities_list =[] # 四半期報告書のリスト

    print('EDINETへのアクセスを開始')
    for i,day in enumerate(day_term):
        print('hoge')
        url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
        params = {"date": day, "type": 2}
        
        # 進捗表示
        if i % 50 == 0:
            print(f'{i}日目：{day}を開始')
        
        # EDINETから1日の書類一覧を取得
        res = requests.get(url, params=params, verify=False)
        
        print(res)
        
        # 必要な書類の項目を抜き出し
        if res.ok:
            json_data = res.json() 
            a = json_data['metadata']['status']
            if int(a) == 200:                          
                for data in json_data['results']:
                    # 指定した会社の指定した書類を抜き出し
                    print(data)
                    name = None
                    if data['edinetCode'] is not None:
                        edinetCode = data['edinetCode']
                        if data['ordinanceCode'] == ordinance_code and data['formCode'] == form_code_quart and edinetCode in company_list:
                            quart_list.append(data)
                        elif data['ordinanceCode'] == ordinance_code and data['formCode'] == form_code_securities and edinetCode in company_list:
                            securities_list.append(data)
                    else:
                        print('fuga')
            else:
                print(f'エラー{day},{a}')       
        else:
            print(f'アクセス失敗かも{day}')
    
    list_dic = {'四半期報告書':quart_list,'有価証券報告書':securities_list} 
    
    return list_dic

In [3]:
def get_zip(list_dic,quart_dir_path,securities_dir_path):
    '''取得したいデータをzipファイルで取得してファイルパスのリストを返す'''
    
    dir_path_dic = {'四半期報告書':quart_dir_path,'有価証券報告書':securities_dir_path}
    file_path_dic = {'四半期報告書':[],'有価証券報告書':[]} # ダウンロードした有価証券報告書のパスを格納する辞書   

    for key in list_dic.keys():
        
        # すでにzipをDLしている場合のため、既存のdocIDリストを取得
        files = os.listdir(dir_path_dic[key])

        existing_docID_list = [file.split('.')[0].split('_')[2] for file in files if os.path.isfile(os.path.join(dir_path_dic[key], file))]
        
        print(f'{key}ファイルのDLを開始')
        for i, doc in enumerate(list_dic[key]):
            # zipファイルパスのリストの作成
            file_name = doc['filerName'].replace('株式会社', '').replace('ホールディングス','ＨＬＤＧ').replace('\xa0','').replace('\u3000','').replace('\n','').replace(' ','') + '_' + doc['edinetCode'] + '_' + doc['docID']
            file_path =  os.path.join(dir_path_dic[key], file_name + ".zip")
            file_path_dic[key].append(file_path)
            
            # 所有していないファイルの場合はDLを行う
            if doc['docID'] not in existing_docID_list:
                # ファイルを取得
                url_zip = "https://disclosure.edinet-fsa.go.jp/api/v1/documents/" + doc['docID']
                params_zip = {"type": 1}

                # 進捗表示
                if i % 100 == 0:
                    print(f'{i}ファイル目を開始')

                # データのDL
                res_zip = requests.get(url_zip, params=params_zip, verify=False, stream=True)

                # zipとして保存
                if res_zip.status_code == 200:
                    with open(file_path, 'wb') as f:
                        for chunk in res_zip.iter_content(chunk_size=1024):
                            if chunk:
                                f.write(chunk)
                                f.flush()

    return file_path_dic

In [4]:
def get_zip_path(quart_dir_path,securities_dir_path,company_df):
    '''取得したいデータファイルパスのリストを返す'''
    
    dir_path_dic = {'四半期報告書':quart_dir_path,'有価証券報告書':securities_dir_path}
    file_path_dic = {'四半期報告書':[],'有価証券報告書':[]} # ダウンロードした有価証券報告書のパスを格納する辞書   
    
    # 会社コードリストを取得
    company_list =list(company_df['ＥＤＩＮＥＴコード'])
    
    for key in dir_path_dic.keys():
        # すでにzipをDLしている場合のため、既存のdocIDリストを取得
        files = os.listdir(dir_path_dic[key])
        existing_file_list = [file for file in files if os.path.isfile(os.path.join(dir_path_dic[key], file))] 
        file_path_dic[key] = [os.path.join(dir_path_dic[key], file) for file in existing_file_list if file.split('.')[0].split('_')[1] in company_list]

    return file_path_dic

In [5]:
def zip_to_df(file_path_dic):
    '''ダウンロードしたzipをdfに変換して各会社のdicにして返す'''
    
    all_df_dic = {}
    for key in file_path_dic.keys():
        print(f'{key}データの変換を開始')        
        df_dic = {}
        for i,company_zip in enumerate(file_path_dic[key]):
            # 進捗表示
            if i % 100 == 0:
                print(f'{i}ファイル目を開始')
            company_name = os.path.splitext(os.path.basename(company_zip))[0].split('_')[0]
            edinet_code = os.path.splitext(os.path.basename(company_zip))[0].split('_')[1]
            doc_name = os.path.splitext(os.path.basename(company_zip))[0].split('_')[2]

            if edinet_code not in df_dic:# 会社コードが辞書中に存在しない場合
                df_dic[edinet_code] = {}
                df_dic[edinet_code] = read_xbrl_from_zip(company_zip)[0]
            elif edinet_code in df_dic:# 会社が辞書中に存在する場合
                df_dic[edinet_code] = pd.concat( [df_dic[edinet_code],read_xbrl_from_zip(company_zip)[0]])
        
        all_df_dic[key] = df_dic

    return all_df_dic

In [6]:
def pickup_shareholder(all_df_dic,company_df):
    '''社長の持ち株比率を取得'''
    
    company_df = company_df.set_index('ＥＤＩＮＥＴコード')
    company_df['社長名'] = np.nan
    company_df['最大株主'] = np.nan
    company_df['社長持ち株比率%'] = 0

    
    for company in list(company_df.index):
        if company in all_df_dic['有価証券報告書'].keys():
            # 必要な有価証券報告書を抽出
            securities = all_df_dic['有価証券報告書'][company]
            new_securities = securities[securities['提出日'] == max(securities['提出日'].unique()) ]#最新の有価証券報告書のみを抽出
            # 株主名簿を取得
            NameMajorShareholders = [name.replace('\xa0','').replace('\u3000','').replace('\n','').replace(' ','')  for name in list(new_securities[new_securities['tag'] == 'NameMajorShareholders']['値'])]
            # 株主の保有率を取得
            share_no = [no  for no in list(new_securities[new_securities['tag'] == 'NameMajorShareholders']['context'])]
            # 各株主のshare ratioを取得
            share_ratio_list = []
            for no in share_no:
                share_ratio_list.append(float(new_securities[(new_securities['context'] == no) & (new_securities['tag'] == 'ShareholdingRatio')]['値']) *100)
            # 社長の名前を取得
            securities = all_df_dic['有価証券報告書'][company]
            temp = [name.replace('\xa0','').replace('\u3000','').replace('\n','').replace(' ','')  for name in list(new_securities[(new_securities['context'] == 'FilingDateInstant')&(new_securities['tag'] == 'TitleAndNameOfRepresentativeCoverPage')]['値'])]
            if not temp:
                temp = list(new_securities[(new_securities['tag'] == 'NameInformationAboutDirectorsAndCorporateAuditors')]['値'])

                if temp:
                    title_name = temp[0].replace('\xa0','').replace('\u3000','').replace('\n','').replace(' ','')
                else:
                    title_name = None
            else:
                title_name = temp[0]

            # 社長の持ち株比率を取得
            for i,(name, ratio) in enumerate(zip(NameMajorShareholders,share_ratio_list)):
                if i == 0:
                    company_df.loc[company,'最大株主'] = name
                
                if title_name: 
                    company_df.loc[company,'社長名'] = title_name
                    
                    if title_name.endswith(name):                       
                        company_df.loc[company,'社長持ち株比率%'] = ratio
                else:
                    pass
    
    return company_df

In [7]:
# main部分----------------------------------------------------------------------
# コードリスト
code_company_df = pd.read_csv('./data/EdinetcodeDlInfo.csv', header=1, encoding='cp932')
code_company_df = code_company_df.rename({'証券コード':'コード','提出者名':'会社名'},axis=1)
code_company_df = code_company_df[code_company_df['上場区分'] == '上場']
code_company_df['コード'] = code_company_df['コード']/10

In [9]:
code_company_df.head(5)

,ＥＤＩＮＥＴコード,提出者種別,上場区分,連結の有無,資本金,決算日,会社名,提出者名（英字）,提出者名（ヨミ）,所在地,提出者業種,コード,提出者法人番号
0,E00004,内国法人・組合,上場,有,1491.0,5月31日,カネコ種苗株式会社,"KANEKO SEEDS CO., LTD.",カネコシュビョウカブシキガイシャ,前橋市古市町一丁目５０番地１２,水産・農林業,1376.0,5.070001e+12
1,E00006,内国法人・組合,上場,有,13500.0,5月31日,株式会社 サカタのタネ,SAKATA SEED CORPORATION,カブシキガイシャ サカタノタネ,横浜市都筑区仲町台２－７－１,水産・農林業,1377.0,6.020001e+12
2,E00007,内国法人・組合,上場,有,100.0,3月31日,株式会社雪国まいたけ,"YUKIGUNI MAITAKE CO.,LTD.",カブシキカイシャユキグニマイタケ,南魚沼市余川８９番地,水産・農林業,1375.0,1.010001e+12
3,E00008,内国法人・組合,上場,有,5500.0,3月31日,ホクト株式会社,HOKUTO CORPORATION,ホクトカブシキガイシャ,長野市南堀１３８番地１,水産・農林業,1379.0,6.100001e+12
4,E00009,内国法人・組合,上場,有,452.0,6月30日,株式会社アクシーズ,"AXYZ Co., Ltd",カブシキガイシャアクシーズ,鹿児島市草牟田二丁目１番８号,水産・農林業,1381.0,5.340001e+12


In [40]:
# 会社のリストを読み込み
company_list = code_company_df

In [41]:
company_list

,ＥＤＩＮＥＴコード,提出者種別,上場区分,連結の有無,資本金,決算日,会社名,提出者名（英字）,提出者名（ヨミ）,所在地,提出者業種,コード,提出者法人番号
0,E00004,内国法人・組合,上場,有,1491.0,5月31日,カネコ種苗株式会社,"KANEKO SEEDS CO., LTD.",カネコシュビョウカブシキガイシャ,前橋市古市町一丁目５０番地１２,水産・農林業,1376.0,5.070001e+12
1,E00006,内国法人・組合,上場,有,13500.0,5月31日,株式会社 サカタのタネ,SAKATA SEED CORPORATION,カブシキガイシャ サカタノタネ,横浜市都筑区仲町台２－７－１,水産・農林業,1377.0,6.020001e+12
2,E00007,内国法人・組合,上場,有,100.0,3月31日,株式会社雪国まいたけ,"YUKIGUNI MAITAKE CO.,LTD.",カブシキカイシャユキグニマイタケ,南魚沼市余川８９番地,水産・農林業,1375.0,1.010001e+12
3,E00008,内国法人・組合,上場,有,5500.0,3月31日,ホクト株式会社,HOKUTO CORPORATION,ホクトカブシキガイシャ,長野市南堀１３８番地１,水産・農林業,1379.0,6.100001e+12
4,E00009,内国法人・組合,上場,有,452.0,6月30日,株式会社アクシーズ,"AXYZ Co., Ltd",カブシキガイシャアクシーズ,鹿児島市草牟田二丁目１番８号,水産・農林業,1381.0,5.340001e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9664,E36077,内国法人・組合,上場,有,200.0,11月末日,ＭＩＴホールディングス株式会社,"MIT Holdings CO.,LTD.",ミットホールディングスカブシキガイシャ,千葉市美浜区中瀬二丁目６番地１,情報・通信業,4016.0,8.040001e+12
9665,E36078,内国法人・組合,上場,無,72.0,2月末日,株式会社ジオコード,"GEOCODE CO., Ltd.",カブシキガイシャジオコード,新宿区新宿四丁目１番６号,サービス業,7357.0,4.010401e+12
9682,E36100,内国法人・組合,上場,有,100.0,2月末日,株式会社クリーマ,NaN,カブシキガイシャクリーマ,港区北青山２ー１２ー５ ＫＲＴビル２Ｆ,情報・通信業,4017.0,6.011001e+12
9705,E36140,内国法人・組合,上場,有,102.0,12月末日,株式会社ビーイングホールディングス,NaN,ガブシキカイシャビーイングホールディングス,金沢市専光寺町レ３番地１８,陸運業,NaN,8.220001e+12


In [43]:
# 取得期間の設定：直近n日分
delta_day = 5
end = date.today()
start = date.today() - timedelta(days=delta_day)

In [44]:
# ダウンロードしたデータのフォルダパス
quart_dir_path = r'./data/output/quart_report'
securities_dir_path = r'./data/output/securities_report'

In [45]:
# XBRLデータの取得 
list_dic = get_list(start,end,company_list)

EDINETへのアクセスを開始
hoge
0日目：2020-12-28を開始
<Response [200]>
{'seqNumber': 1, 'docID': 'S100JNB2', 'edinetCode': 'E12444', 'secCode': None, 'JCN': '8010001114914', 'filerName': '三井住友トラスト・アセットマネジメント株式会社', 'fundCode': 'G08643', 'ordinanceCode': '030', 'formCode': '10A000', 'docTypeCode': '160', 'periodStart': '2020-04-01', 'periodEnd': '2021-03-31', 'submitDateTime': '2020-12-28 09:00', 'docDescription': '半期報告書（内国投資信託受益証券）－第19期(令和2年4月1日－令和3年3月31日)', 'issuerEdinetCode': None, 'subjectEdinetCode': None, 'subsidiaryEdinetCode': None, 'currentReportReason': None, 'parentDocID': None, 'opeDateTime': None, 'withdrawalStatus': '0', 'docInfoEditStatus': '0', 'disclosureStatus': '0', 'xbrlFlag': '1', 'pdfFlag': '1', 'attachDocFlag': '0', 'englishDocFlag': '0'}
{'seqNumber': 2, 'docID': 'S100KFR2', 'edinetCode': 'E32255', 'secCode': None, 'JCN': None, 'filerName': 'Ｂｌａｃｋ\u3000Ｃｌｏｖｅｒ\u3000Ｌｉｍｉｔｅｄ', 'fundCode': None, 'ordinanceCode': '060', 'formCode': '010002', 'docTypeCode': '350', 'periodStart': None,

<Response [200]>
hoge
<Response [200]>
hoge
<Response [200]>
hoge
<Response [200]>


In [46]:
print(list_dic)

{'四半期報告書': [{'seqNumber': 82, 'docID': 'S100KG6Y', 'edinetCode': 'E03081', 'secCode': '82760', 'JCN': '3160001008726', 'filerName': '株式会社\u3000平和堂', 'fundCode': None, 'ordinanceCode': '010', 'formCode': '043000', 'docTypeCode': '140', 'periodStart': '2020-08-21', 'periodEnd': '2020-11-20', 'submitDateTime': '2020-12-28 09:17', 'docDescription': '四半期報告書－第64期第3四半期(令和2年8月21日－令和2年11月20日)', 'issuerEdinetCode': None, 'subjectEdinetCode': None, 'subsidiaryEdinetCode': None, 'currentReportReason': None, 'parentDocID': None, 'opeDateTime': None, 'withdrawalStatus': '0', 'docInfoEditStatus': '0', 'disclosureStatus': '0', 'xbrlFlag': '1', 'pdfFlag': '1', 'attachDocFlag': '0', 'englishDocFlag': '0'}, {'seqNumber': 104, 'docID': 'S100KGF0', 'edinetCode': 'E01856', 'secCode': '69050', 'JCN': '7230001000928', 'filerName': 'コーセル株式会社', 'fundCode': None, 'ordinanceCode': '010', 'formCode': '043000', 'docTypeCode': '140', 'periodStart': '2020-08-21', 'periodEnd': '2020-11-20', 'submitDateTime': '2020-12-

In [47]:
file_path_dic = get_zip(list_dic,quart_dir_path,securities_dir_path)

四半期報告書ファイルのDLを開始
0ファイル目を開始
有価証券報告書ファイルのDLを開始
0ファイル目を開始


In [49]:
# XBRLからデータ形式を変換
df_dic = zip_to_df(file_path_dic)

四半期報告書データの変換を開始
0ファイル目を開始
有価証券報告書データの変換を開始
0ファイル目を開始


In [57]:
quart = df_dic['四半期報告書']['E03081']

In [59]:
quart_oi = quart[(quart['tag'] == 'OperatingIncome') & (quart['context'] == 'CurrentYTDDuration')]

In [77]:
# 現在の設定確認
print(pd.get_option("display.max_rows"))
pd.set_option('display.max_rows', 400)

150


In [78]:
#quart[['tag', '値']].head(150)
quart.head(400)

,version,提出日,提出回数,報告対象期間期末日,追番,第N期,名前空間接頭辞,tag,id,context,開始日,終了日,期末日,連結,値
0,2,2020-12-28,1,2020-11-20,0,3,jpdei_cor,NumberOfSubmissionDEI,None,FilingDateInstant,NaT,NaT,2020-12-28,True,1
1,2,2020-12-28,1,2020-11-20,0,3,jpdei_cor,EDINETCodeDEI,None,FilingDateInstant,NaT,NaT,2020-12-28,True,E03081
2,2,2020-12-28,1,2020-11-20,0,3,jpdei_cor,FundCodeDEI,None,FilingDateInstant,NaT,NaT,2020-12-28,True,None
3,2,2020-12-28,1,2020-11-20,0,3,jpdei_cor,SecurityCodeDEI,None,FilingDateInstant,NaT,NaT,2020-12-28,True,82760
4,2,2020-12-28,1,2020-11-20,0,3,jpdei_cor,FilerNameInJapaneseDEI,None,FilingDateInstant,NaT,NaT,2020-12-28,True,株式会社 平和堂
5,2,2020-12-28,1,2020-11-20,0,3,jpdei_cor,FilerNameInEnglishDEI,None,FilingDateInstant,NaT,NaT,2020-12-28,True,"HEIWADO CO.,LTD."
6,2,2020-12-28,1,2020-11-20,0,3,jpdei_cor,FundNameInJapaneseDEI,None,FilingDateInstant,NaT,NaT,2020-12-28,True,None
7,2,2020-12-28,1,2020-11-20,0,3,jpdei_cor,FundNameInEnglishDEI,None,FilingDateInstant,NaT,NaT,2020-12-28,True,None
8,2,2020-12-28,1,2020-11-20,0,3,jpdei_cor,CabinetOfficeOrdinanceDEI,None,FilingDateInstant,NaT,NaT,2020-12-28,True,企業内容等の開示に関する内閣府令
9,2,2020-12-28,1,2020-11-20,0,3,jpdei_cor,DocumentTypeDEI,None,FilingDateInstant,NaT,NaT,2020-12-28,True,第四号の三様式
